In [2]:
import pandas as pd
import numpy as np
import pickle

In [12]:
rat = pd.read_csv('ratings_small.csv')
rat.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [13]:
len(rat)

100004

In [14]:
#number of unique movies in the data

m=len(rat.movieId.unique())
m

9066

The observation showed that the ratings_small.csv has already been filtered and left with the users that did at least 20 ratings. So to split it into train and test sets, we will use the timestamp to fill the newest ones in the test set.

In [15]:
#number of unique users who rated movies

n=len(rat.userId.unique())
n

671

In [16]:
#determining which movie is in which column of my soon-to-be CF matrix and vice versa

mat_mov_dict=dict(zip(np.arange(m),rat.movieId.unique()))
mov_mat_dict=dict(zip(rat.movieId.unique(),np.arange(m)))

In [17]:
#making an all-zero matrix to fill, dimensions are number of movies by number of users

ratings=np.zeros((n,m))

In [11]:
for row in range(rat.shape[0]):
    ratings[int(rat.loc[row].userId-1),mov_mat_dict[int(rat.loc[row].movieId)]]=rat.loc[row].rating
    if row%10000==0:
        print(row)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [12]:
ratings

array([[ 2.5,  3. ,  3. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]])

In [15]:
np.save("ratings_matrix", ratings, allow_pickle=True)

Now the same for the train set

In [19]:
rat_by_time = rat.sort_values(by=['timestamp'], ascending=False)
rat_train = rat_by_time.iloc[1001:]
rat_test = rat_by_time.head(1000)

In [23]:
rat_test.to_csv('ratings_test.csv')

In [20]:
tm=len(rat_train.movieId.unique())
tn=len(rat_train.userId.unique())

[tn, tm]

[666, 8991]

In [21]:
ratings_train=np.zeros((tn,tm))

In [22]:
mat_mov_dict_train=dict(zip(np.arange(tm),rat_train.movieId.unique()))
mov_mat_dict_train=dict(zip(rat_train.movieId.unique(),np.arange(tm)))
user_mat_dict_train=dict(zip(rat_train.userId.unique(),np.arange(tn)))
mat_user_dict_train=dict(zip(np.arange(tn),rat_train.userId.unique()))

In [34]:
for row in range(rat_train.shape[0]):
    ratings_train[user_mat_dict_train[int(rat_train.iloc[row].userId)],mov_mat_dict_train[int(rat_train.iloc[row].movieId)]]=rat_train.iloc[row].rating
    if row%10000==0:
        print(row)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [35]:
ratings_train

array([[ 3. ,  3. ,  1.5, ...,  0. ,  0. ,  0. ],
       [ 2.5,  4. ,  3.5, ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  3.5, ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]])

In [36]:
np.save("ratings_train_matrix", ratings_train, allow_pickle=True)

In [3]:
mov_train = np.load(open('ratings_train_matrix.npy','rb'))
mov_train

array([[ 3. ,  3. ,  1.5, ...,  0. ,  0. ,  0. ],
       [ 2.5,  4. ,  3.5, ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  3.5, ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]])

In [4]:
def similarity(i, j):
    rated_movies=[] #We store the users who rated both i and j movies
    for u in range(len(i)):
        if i[u]!=0 and j[u]!=0:
            rated_movies.append(u)
    
    if len(rated_movies)==0:
        return 0
    Ri = i[rated_movies]
    Rj = j[rated_movies]
    Ri_mean = np.mean(Ri)
    Rj_mean = np.mean(Rj)
    Ri -= Ri_mean
    Rj -= Rj_mean
    
    sim = (Ri.T.dot(Rj))/((np.linalg.norm(Ri))*(np.linalg.norm(Rj)))
    
    return sim

Item-based

In [5]:
sim_train = np.load('train_mov_sim.npy')
sim_train

array([[ 1.        ,  0.        ,  0.04469902, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.50837438, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04469902,  0.50837438,  1.        , ...,         nan,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,         nan, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [6]:
where_are_NaNs = np.isnan(sim_train)
sim_train[where_are_NaNs] = 0

In [7]:
def predictor(movie, user):
    pred_nom=0
    sum_sim=0
    for i in range(len(sim_train[:,movie])):
        if sim_train[:,movie][i]>0:
            pred_nom+=sim_train[:,movie][i]*mov_train[user,:][i]
            sum_sim+=sim_train[:,movie][i]
    if sum_sim!=0:
        return pred_nom/sum_sim
    return 0

In [8]:
mov_test = pd.read_csv('ratings_test.csv')
mov_test.head()

,Unnamed: 0,userId,movieId,rating,timestamp
0,35069,251,1704,4.5,1476640644
1,35096,251,6377,4.5,1476623300
2,35140,251,81847,4.5,1476623282
3,35172,251,134853,5.0,1476623217
4,35125,251,56587,5.0,1476623131


In [23]:
for i in range(mov_test.shape[0]):
    if int(mov_test.iloc[i].userId) in user_mat_dict_train and int(mov_test.iloc[i].movieId) in mov_mat_dict_train: 
        predicted_rating = predictor(mov_mat_dict_train[int(mov_test.iloc[i].movieId)], user_mat_dict_train[int(mov_test.iloc[i].userId)])
        print("predicted rating is: ", predicted_rating, "actual rating is: ", mov_test.iloc[i].rating)    

predicted rating is:  3.06036172775 actual rating is:  1.0
predicted rating is:  0.972967956417 actual rating is:  2.5
predicted rating is:  0.0 actual rating is:  2.5
predicted rating is:  1.88614807891 actual rating is:  3.0
predicted rating is:  0.0 actual rating is:  3.5
predicted rating is:  2.66362341621 actual rating is:  3.5
predicted rating is:  0.104023262493 actual rating is:  5.0
predicted rating is:  0.0860407461289 actual rating is:  3.5
predicted rating is:  0.121260980511 actual rating is:  5.0
predicted rating is:  0.0634377431568 actual rating is:  5.0
predicted rating is:  0.0380631753261 actual rating is:  5.0
predicted rating is:  0.0539612357731 actual rating is:  4.5
predicted rating is:  0.0570698960147 actual rating is:  4.5
predicted rating is:  0.0433028251582 actual rating is:  5.0
predicted rating is:  0.0450261938395 actual rating is:  4.0
predicted rating is:  0.0600840380251 actual rating is:  3.0
predicted rating is:  0.057670541913 actual rating is:  4

predicted rating is:  0.215874195881 actual rating is:  2.0
predicted rating is:  0.150393731833 actual rating is:  1.0
predicted rating is:  0.507782992524 actual rating is:  1.5
predicted rating is:  0.216229855052 actual rating is:  2.5
predicted rating is:  0.283152725364 actual rating is:  3.0
predicted rating is:  0.231209369838 actual rating is:  3.5
predicted rating is:  0.537412538161 actual rating is:  3.0
predicted rating is:  0.616135733531 actual rating is:  3.5
predicted rating is:  0.80805102036 actual rating is:  3.5
predicted rating is:  0.710537557556 actual rating is:  4.0
predicted rating is:  1.3333734412 actual rating is:  4.0
predicted rating is:  0.58816050699 actual rating is:  2.0
predicted rating is:  0.167133700674 actual rating is:  0.5
predicted rating is:  0.108117803398 actual rating is:  0.5
predicted rating is:  0.110425939026 actual rating is:  1.5
predicted rating is:  0.468660721727 actual rating is:  1.5
predicted rating is:  0.0 actual rating is: 

User-based

In [24]:
sim_user_train = np.load('train_user_sim.npy')
sim_user_train

array([[ 1.        ,  0.20652293,  0.22831431, ...,  0.00273254,
         0.10225945,  0.54260952],
       [ 0.20652293,  1.        ,  0.20485888, ...,  0.37578721,
         0.34581105,  0.52595024],
       [ 0.22831431,  0.20485888,  1.        , ...,  0.11216691,
         0.29218277,  0.29567202],
       ..., 
       [ 0.00273254,  0.37578721,  0.11216691, ...,  1.        ,
         0.21114977,  0.68516016],
       [ 0.10225945,  0.34581105,  0.29218277, ...,  0.21114977,
         1.        ,  0.6228411 ],
       [ 0.54260952,  0.52595024,  0.29567202, ...,  0.68516016,
         0.6228411 ,  1.        ]])

In [25]:
where_are_NaNs = np.isnan(sim_user_train)
sim_user_train[where_are_NaNs] = 0

In [26]:
def predictor(movie, user):
    pred_nom=0
    sum_sim=0
    for i in range(len(sim_user_train[:,movie])):
        if sim_user_train[:,movie][i]>0:
            pred_nom+=sim_user_train[user,:][i]*mov_train[:,movie][i]
            sum_sim+=sim_user_train[:,movie][i]
    if sum_sim!=0:
        return pred_nom/sum_sim
    return 0

In [27]:
mov_test = pd.read_csv('ratings_test.csv')
mov_test.head()

,Unnamed: 0,userId,movieId,rating,timestamp
0,35069,251,1704,4.5,1476640644
1,35096,251,6377,4.5,1476623300
2,35140,251,81847,4.5,1476623282
3,35172,251,134853,5.0,1476623217
4,35125,251,56587,5.0,1476623131


In [30]:
for i in range(mov_test.shape[0]):
    if int(mov_test.iloc[i].userId) in user_mat_dict_train and int(mov_test.iloc[i].movieId) in mov_mat_dict_train: 
        predicted_rating = predict_user(mov_mat_dict_train[int(mov_test.iloc[i].movieId)], user_mat_dict_train[int(mov_test.iloc[i].userId)])
        print("predicted rating is: ", predicted_rating, "actual rating is: ", mov_test.iloc[i].rating)   

predicted rating is:  4.10695687977 actual rating is:  1.0
predicted rating is:  3.50176485364 actual rating is:  2.5
predicted rating is:  4.5 actual rating is:  2.5
predicted rating is:  3.52120899606 actual rating is:  3.0
predicted rating is:  3.0 actual rating is:  3.5
predicted rating is:  3.44305375978 actual rating is:  3.5
predicted rating is:  4.5249901571 actual rating is:  5.0
predicted rating is:  4.21609511636 actual rating is:  3.5
predicted rating is:  6.79331165788 actual rating is:  5.0
predicted rating is:  4.06745331315 actual rating is:  5.0
predicted rating is:  4.06287932447 actual rating is:  5.0
predicted rating is:  4.04436214259 actual rating is:  4.5
predicted rating is:  3.79155496645 actual rating is:  4.5
predicted rating is:  3.68898745552 actual rating is:  5.0
predicted rating is:  3.66263886355 actual rating is:  4.0
predicted rating is:  3.88523501181 actual rating is:  3.0
predicted rating is:  3.56242105827 actual rating is:  4.0
predicted rating i

predicted rating is:  4.32358054114 actual rating is:  3.5
predicted rating is:  4.01134610304 actual rating is:  3.5
predicted rating is:  3.83724763691 actual rating is:  3.5
predicted rating is:  4.21738951969 actual rating is:  4.0
predicted rating is:  3.44525227582 actual rating is:  3.5
predicted rating is:  3.75935870306 actual rating is:  3.5
predicted rating is:  4.55807896489 actual rating is:  3.5
predicted rating is:  4.39729187773 actual rating is:  2.0
predicted rating is:  3.84699216962 actual rating is:  4.0
predicted rating is:  3.78720561861 actual rating is:  3.5
predicted rating is:  1.55322488764 actual rating is:  1.5
predicted rating is:  2.50077307142 actual rating is:  1.5
predicted rating is:  2.87391245542 actual rating is:  1.0
predicted rating is:  2.1074962972 actual rating is:  1.0
predicted rating is:  2.53683393661 actual rating is:  0.5
predicted rating is:  1.42517208816 actual rating is:  1.5
predicted rating is:  3.17354547611 actual rating is:  1.

In [29]:
def predict_user(movie,user):
    pred_nom=0
    sum_sim=0
    for i in range(len(mov_train[:,movie])):
        if mov_train[:,movie][i]!=0:
            pred_nom+=mov_train[:,movie][i]*sim_user_train[user,i]
            sum_sim+=sim_user_train[user,i]
    if sum_sim!=0:
        return pred_nom/sum_sim
    return 0